In [1]:
# Neccesary imports
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from time import time
import random
import glob, os

from sklearn.model_selection import train_test_split

# from scipy.ndimage.filters import gaussian_filter1d
# import torchvision.models as models
# from tqdm import tqdm
# from sklearn.metrics import mean_squared_error
# import matplotlib.pyplot as plt
# from accelerate import Accelerator
# cm = plt.get_cmap('RdYlBu')
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from torch.nn import LayerNorm
# import math
# from sklearn.model_selection import train_test_split
# import seaborn as sns
# import matplotlib.colors as mcolors
# import pandas as pd

# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# from scipy.stats import gaussian_kde

In [2]:
%%time
# Import all the data
data = pd.read_pickle('grism_specPT.pkl')

# pulling only datapoints with a SNR at or above 2.5 and a redshift below 1.7
data = data[data['SNR']>=2.5]
data_subset = data[data['z']<1.7]
data_subset.head()

CPU times: user 1.56 s, sys: 2.63 s, total: 4.19 s
Wall time: 18.8 s


,grism_id,wavelength,flux,z,SNR,continuum_sub_flux
2,aegis-26-G141_00469,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.43,4.094828,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,aegis-26-G141_00703,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.40,20.695364,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
13,aegis-26-G141_00836,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.56,6.308725,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
16,aegis-26-G141_00910,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.05,4.627907,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
18,aegis-26-G141_00937,"[10208.409432389317, 10209.33721533346, 10210....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.51,8.418367,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [3]:
# Split the data into 70% train and 30% temp_test
train_df, temp_test_df = train_test_split(data_subset, test_size=0.3, random_state=42)

# Split the temp_test into 50% test and 50% validation, which is 15% each of the original
test_df, val_df = train_test_split(temp_test_df, test_size=0.5, random_state=42)

# Print the sizes of each set
print(f'Train set size: {len(train_df)}')
print(f'Validation set size: {len(test_df)}')
print(f'Test set size: {len(val_df)}')

Train set size: 6078
Validation set size: 1302
Test set size: 1303


## Loading in SpecPT

In [4]:
from SpecPT import (
    SpecPT, 
    SpecPTForRedshift, 
    CustomLoadDataset_Autoencoder,
    Swish,
    # CustomLoadDataset_Redshift, 
    NMADLoss, 
    evaluate
)

In [5]:
class EnhancedSpecPTForRedshift(nn.Module):
    def __init__(self, pretrained_model, output_features=1, num_mlp_blocks=5, mlp_dim=512, dropout_rate=0.2):
        super(EnhancedSpecPTForRedshift, self).__init__()
        
        self.encoder = pretrained_model.transformer_encoder
        self.proj_to_d_model = pretrained_model.proj_to_d_model
        self.forward_conv = pretrained_model.forward_conv
        
        # Fine-tune the last few layers of the encoder
        for param in list(self.encoder.parameters())[-4:]:
            param.requires_grad = True
        
        self.mlp_blocks = nn.Sequential(
            *[ImprovedResidualMLPBlock(mlp_dim if i > 0 else 512, mlp_dim, dropout_rate) for i in range(num_mlp_blocks)]
        )
        
        self.prediction = nn.Sequential(
            nn.Linear(mlp_dim, mlp_dim // 2),
            Swish(),
            nn.Dropout(dropout_rate),
            nn.Linear(mlp_dim // 2, output_features),
            nn.Softplus()
        )
        
        self.attention = nn.MultiheadAttention(embed_dim=512, num_heads=8)
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.forward_conv(x)
        x = x.flatten(start_dim=1)
        x = self.proj_to_d_model(x)
        x = x.unsqueeze(0)
        
        encoded_features = self.encoder(x)
        encoded_features = encoded_features.squeeze(0)
        
        # Apply attention mechanism
        attn_output, _ = self.attention(encoded_features, encoded_features, encoded_features)
        x = attn_output + encoded_features  # Residual connection
        
        x = self.mlp_blocks(x)
        redshift = self.prediction(x)
        return redshift

class ImprovedResidualMLPBlock(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate):
        super(ImprovedResidualMLPBlock, self).__init__()
        self.linear1 = nn.Linear(input_dim, output_dim)
        self.linear2 = nn.Linear(output_dim, output_dim)
        self.swish = Swish()
        self.layer_norm = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        residual = x
        x = self.swish(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        x = x + residual  # Residual connection
        x = self.layer_norm(x)
        return self.swish(x)


In [6]:
# Dataset Loader for Redshift
class CustomLoadDataset_Redshift(Dataset):
    def __init__(self, df):
        x = []
        y = []
        target_id = []
            
        for index, row in df.iterrows():
            fl = row['flux']
            if np.median(fl) > 0:
                fl = fl / np.median(fl)
                x.append(fl)
                y.append(np.array([row['z']]))
                target_id.append(row['grism_id'])

        self.X = torch.from_numpy(np.stack(x, axis=0))
        self.Y = torch.from_numpy(np.stack(y, axis=0))
        self.t_id = target_id

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].float(), self.Y[idx].float(), idx, self.t_id[idx]


In [7]:
valid_loader = DataLoader(CustomLoadDataset_Redshift(val_df), batch_size=100, shuffle=True)
test_loader = DataLoader(CustomLoadDataset_Redshift(test_df), batch_size=256, shuffle=False)

In [8]:
# defining our model
model = SpecPT(input_size=7781) #, d_head=16

/home/fm5957/miniconda3/envs/ml/lib/python3.9/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [9]:
model.transformer_decoder.layers

ModuleList(
  (0-2): 3 x TransformerDecoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (dropout3): Dropout(p=0.1, inplace=False)
  )
)

Following the tutorial from [Cam Metrics and Training Tutorial](https://github.com/jacobgil/pytorch-grad-cam/blob/master/tutorials/CAM%20Metrics%20And%20Tuning%20Tutorial.ipynb)

In [12]:
from pytorch_grad_cam import GradCAM
# targeting the layers of the decoder
target_layers = model.transformer_decoder.layers

# applying the cam to the layers
cam = GradCAM(model, target_layers)

In [14]:
import torchvision.transforms as transforms

# choosing a galaxy example
gal_ex = np.array([train_df.T[17364].wavelength,train_df.T[17364].flux])

transform = transforms.ToTensor()
tensor = transform(gal_ex).unsqueeze(0)

In [16]:
# loading in the input tensor (issue: this seems to only take an image, 
# I am having troubles getting it to take in the 2D spectra)
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

with GradCAM(model=model, target_layers=target_layers) as cam:
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
    cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)
cam = np.uint8(255*grayscale_cams[0, :])
cam = cv2.merge([cam, cam, cam])
images = np.hstack((np.uint8(255*img), cam , cam_image))
Image.fromarray(images)

NameError: name 'preprocess_image' is not defined

In [24]:
targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]
target_layers = [model.backbone]
cam = AblationCAM(model,
                  target_layers, 
                  use_cuda=torch.nn.cuda.is_available(), 
                  reshape_transform=fasterrcnn_reshape_transform,
                  ablation_layer=AblationLayerFasterRCNN(),
                  ratio_channels_to_ablate=1.0)

# or a very fast alternative:

cam = EigenCAM(model,
              target_layers, 
              use_cuda=torch.nn.cuda.is_available(), 
              reshape_transform=fasterrcnn_reshape_transform)


NameError: name 'labels' is not defined